<a href="https://colab.research.google.com/github/Helena0811/DataScience_LOP/blob/master/Bag%20of%20words%20Meets%20Bags%20of%20Popcorn/Movie_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 초기 세팅

In [1]:
import pandas as pd
import numpy as np
import warnings
import os
import matplotlib.pyplot as plt
import seaborn as sns

warnings .filterwarnings('ignore')
SEED = 33

#데이터 로드

In [2]:
#DATA = '/content/Bag of Words Meets Bags of Popcorn/Data'
DATA = 'C:/Users/hyeon/Documents/DataScience-LOP-/Bag of words Meets Bags of Popcorn/Data'

#os.path.join으로 파일 가져옴
train = pd.read_csv(os.path.join(DATA, 'labeledTrainData.tsv'), delimiter='\t')
test = pd.read_csv(os.path.join(DATA, 'testData.tsv'), delimiter='\t')
unlabled_train = pd.read_csv(os.path.join(DATA, 'unlabeledTrainData.tsv'), delimiter='\t', error_bad_lines=False)

b'Skipping line 43043: expected 2 fields, saw 3\n'


In [3]:
print(train.shape)
train.head()

(25000, 3)


,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [4]:
print(test.shape)
test.head()

(25000, 2)


,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


In [5]:
print(unlabled_train.shape)
unlabled_train.head()

(49998, 2)


,id,review
0,9999_0,"Watching Time Chasers, it obvious that it was ..."
1,45057_0,I saw this film about 20 years ago and remembe...
2,15561_0,"Minor Spoilers<br /><br />In New York, Joan Ba..."
3,7161_0,I went to see this film with a great deal of e...
4,43971_0,"Yes, I agree with everyone on this site this m..."


#전처리

In [6]:
#html 태그 제거
from bs4 import BeautifulSoup

#불용어 제거
#자주 쓰이지만 의미없는 단어들!
#해당 소스가 먹히지 않아 download 추가...
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hyeon\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
sample = train['review'][0]

In [8]:
sample

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [9]:
soup = BeautifulSoup(sample, 'html.parser')

In [10]:
#태그 제거됨
soup.text

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 20 mi

In [11]:
#특수기호 제거
import re

#소문자 a-z, A-Z를 제외한 모든 문자는 공백으로 대체
cleaned = re.sub('[^a-zA-Z]', ' ', soup.text)

In [12]:
cleaned

'With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    mi

In [13]:
#모두 소문자로 출력
cleaned.lower()

'with all this stuff going down at the moment with mj i ve started listening to his music  watching the odd documentary here and there  watched the wiz and watched moonwalker again  maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  some of it has subtle messages about mj s feeling towards the press and also the obvious message of drugs are bad m kay visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring  some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him the actual feature film bit when it finally starts is only on for    mi

# Lemmatizer

어간 추출 전처리
ex) run, ran, runs -> run

단어는 하나씩 들어가야 함!

In [122]:
from nltk.stem import WordNetLemmatizer

#그냥 import하니 에러발생, 직접 다운로드 받기
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hyeon\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [123]:
lemmatizer = WordNetLemmatizer()

In [124]:
print(lemmatizer.lemmatize('runs', 'v'))
print(lemmatizer.lemmatize('ran', 'v'))
print(lemmatizer.lemmatize('run', 'v'))

print(lemmatizer.lemmatize('apple', 'v'))
print(lemmatizer.lemmatize('apples', 'v'))

print(lemmatizer.lemmatize('runs'))
print(lemmatizer.lemmatize('apples'))

run
run
run
apple
apples
run
apple


In [125]:
np.array(all_review_clean)[0].split()

['stuff',
 'going',
 'moment',
 'mj',
 'started',
 'listening',
 'music',
 'watching',
 'odd',
 'documentary',
 'watched',
 'wiz',
 'watched',
 'moonwalker',
 'maybe',
 'want',
 'get',
 'certain',
 'insight',
 'guy',
 'thought',
 'really',
 'cool',
 'eighties',
 'maybe',
 'make',
 'mind',
 'whether',
 'guilty',
 'innocent',
 'moonwalker',
 'part',
 'biography',
 'part',
 'feature',
 'film',
 'remember',
 'going',
 'see',
 'cinema',
 'originally',
 'released',
 'subtle',
 'messages',
 'mj',
 'feeling',
 'towards',
 'press',
 'also',
 'obvious',
 'message',
 'drugs',
 'bad',
 'kay',
 'visually',
 'impressive',
 'course',
 'michael',
 'jackson',
 'unless',
 'remotely',
 'like',
 'mj',
 'anyway',
 'going',
 'hate',
 'find',
 'boring',
 'may',
 'call',
 'mj',
 'egotist',
 'consenting',
 'making',
 'movie',
 'mj',
 'fans',
 'would',
 'say',
 'made',
 'fans',
 'true',
 'really',
 'nice',
 'actual',
 'feature',
 'film',
 'bit',
 'finally',
 'starts',
 'minutes',
 'excluding',
 'smooth',
 'crim

In [127]:
def process_lemma(sentence):
    #for word in sentence:
        #w = lemmatizer.lemmatize(word, 'v')
        #ret.append(w)
    ret = [lemmatizer.lemmatize(word, 'v') for word in sentence]
    return ret

In [ ]:
#전처리 함수 만들기
def preprocessing_original(sentence):
  soup = BeautifulSoup(sentence, 'html.parser')
  cleaned = re.sub('[^a-zA-Z]', ' ', soup.text)
  cleaned = cleaned.lower()
  cleaned = [word for word in cleaned.split() if word not in eng_stopwords]
  #현재 processing(sample)을 찍어보면 단어가 분리된 형태로 나오는데, 이를 다시 문장 형태로 만들기
  return ' '.join(cleaned)

In [154]:
#lemmatizer를 활용한 전처리 함수 만들기
def preprocessing(sentence):
  soup = BeautifulSoup(sentence, 'html.parser')
  cleaned = re.sub('[^a-zA-Z]', ' ', soup.text)
  cleaned = cleaned.lower()
  cleaned = [word for word in cleaned.split() if word not in eng_stopwords]
  cleaned = process_lemma(cleaned)
  #현재 processing(sample)을 찍어보면 단어가 분리된 형태로 나오는데, 이를 다시 문장 형태로 만들기
  return ' '.join(cleaned)

In [155]:
s = ['This', 'is', 'a', 'pen']

In [156]:
eng_stopwords = stopwords.words('english')

In [157]:
eng_stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [158]:
#불용어 사전에 포함된 단어 찾기
for word in s:
  if word in eng_stopwords:
    print(word)

is
a


In [159]:
for word in sample.lower().split():
  if word in eng_stopwords:
    print(word)

#아까 만들던 전처리 함수로 가서 이를 적용해보자! Go to In [28]

with
all
this
down
at
the
with
to
his
the
here
and
the
and
i
just
to
a
into
this
who
i
was
in
the
just
to
up
my
he
is
or
is
which
i
to
at
the
when
it
was
some
of
it
has
about
the
and
the
of
are
but
of
this
is
all
about
so
you
in
then
you
are
to
this
and
it
some
an
for
to
the
of
this
but
and
most
of
his
that
he
it
for
the
which
if
is
of
when
it
is
only
on
for
or
so
the
and
is
as
a
all
why
he
so
is
because
his
that
he
to
it
is
he
who
is
so
i
he
just
of
in
this
into
a
and
a
and
the
the
have
had
the
of
a
when
it
to
the
as
with
a
of
them
a
this
is
for
who
on
or
i
is
most
if
then
it
does
and
off
a
and
in
this
is
a
is
of
the
most
to
this
but
is
he
with
all
the
this
i
don't
because
can
be
i
this
for
a
he
is
an
but
or
of
the
most
i
he
is
not
the


In [160]:
preprocessing(sample)

'stuff go moment mj start listen music watch odd documentary watch wiz watch moonwalker maybe want get certain insight guy think really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember go see cinema originally release subtle message mj feel towards press also obvious message drug bad kay visually impressive course michael jackson unless remotely like mj anyway go hate find bore may call mj egotist consent make movie mj fan would say make fan true really nice actual feature film bite finally start minutes exclude smooth criminal sequence joe pesci convince psychopathic powerful drug lord want mj dead bad beyond mj overhear plan nah joe pesci character rant want people know supply drug etc dunno maybe hat mj music lot cool things like mj turn car robot whole speed demon sequence also director must patience saint come film kiddy bad sequence usually directors hate work one kid let alone whole bunch perform complex dance scene botto

In [161]:
#각 데이터 파일에 있는 리뷰 모아서 처리하기!
all_review = pd.concat([train['review'], unlabled_train['review'], test['review']])

print(train['review'][0])
print(unlabled_train['review'][0])
print(test['review'][0])
all_review.head()

With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally star

0    With all this stuff going down at the moment w...
1    \The Classic War of the Worlds\" by Timothy Hi...
2    The film starts with a manager (Nicholas Bell)...
3    It must be assumed that those who praised this...
4    Superbly trashy and wondrously unpretentious 8...
Name: review, dtype: object

In [162]:
#전처리 한 상태의 데이터를 변수에 넣기 - 전처리 하고 저장하는데 오래걸리네ㅠㅠ
all_review_clean = all_review.apply(preprocessing)

In [163]:
#엥 왜 concat이 안된거같지...?
all_review_clean.head()

0    stuff go moment mj start listen music watch od...
1    classic war worlds timothy hines entertain fil...
2    film start manager nicholas bell give welcome ...
3    must assume praise film greatest film opera ev...
4    superbly trashy wondrously unpretentious explo...
Name: review, dtype: object

#CountVectorizer

단어사전을 만든 후 해당 단어에 대한 빈도수를 세어줌

In [164]:
#counter vectorizer
#단어에 대한 갯수를 세서 분석
from sklearn.feature_extraction.text import CountVectorizer

In [165]:
cv = CountVectorizer(analyzer='word', max_features=5000)

In [166]:
all_review_cv = cv.fit_transform(all_review_clean)

In [167]:
all_review_cv.shape

(99998, 5000)

In [168]:
all_review_cv[0].toarray()

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

#Train Test Split

In [169]:
#concat해서 붙인걸 다시 분리
train_sentences = all_review_cv[:len(train)]
test_sentences = all_review_cv[-len(test):]

In [170]:
train_sentences.shape,test_sentences.shape

((25000, 5000), (25000, 5000))

In [171]:
train_lables = train['sentiment']
train_lables.shape

(25000,)

#Model

In [172]:
from sklearn.ensemble import RandomForestClassifier

In [173]:
#random_state로 성능을 동일하게 만들어줌
rfc = RandomForestClassifier(n_estimators=1000, max_depth=8, n_jobs=-1, random_state=SEED)

In [174]:
rfc.fit(train_sentences, train_lables)

RandomForestClassifier(max_depth=8, n_estimators=1000, n_jobs=-1,
                       random_state=33)

In [175]:
prediction = rfc.predict(test_sentences)

In [176]:
prediction.shape

(25000,)

In [177]:
prediction[:10]

array([1, 0, 1, 1, 1, 0, 0, 1, 0, 1], dtype=int64)

#Submission

In [178]:
submission = pd.read_csv(os.path.join(DATA, 'sampleSubmission.csv'))

In [179]:
submission.head()

,id,sentiment
0,12311_10,0
1,8348_2,0
2,5828_4,0
3,7186_2,0
4,12128_7,0


In [180]:
submission['sentiment'] = prediction

In [181]:
submission['sentiment'].value_counts()

1    13835
0    11165
Name: sentiment, dtype: int64

In [182]:
import datetime

In [183]:
timestring = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')

In [184]:
filename = f'Submission/submission-{timestring}.csv'

In [185]:
filename

'Submission/submission-2020-10-24-23-09-25.csv'

In [186]:
submission.to_csv(filename, index=False)